In [ ]:
import numpy as np
import lmfit

from pathlib import Path

In [ ]:
# Configuration
potentials = ["an_cai", "da1p"]
# states = ["gs", "0.32", "1.78", "2.65", "3.40n", "3.40p"]
states = ["gs", "gsn", "0.32", "0.32p", "1.78", "1.78n", "2.65", "2.65p", "3.40n", "3.40p"]
energies = ["mid"]
theory_dir = Path("C:\\Users\\zachs\\Desktop\\data4plots\\theory_calculations")
# theory_dir = Path("C:\\Users\\zachs\\Desktop\\ptolemy_calculations")
xs_dir = Path("C:\\Users\\zachs\\Desktop\\data4plots\\xs_gate_err\\xs_with_err")
# write_path = Path("C:\\Users\\zachs\\Desktop")

In [ ]:
skip = ["gsn", "0.32p", "1.78n", "2.65p", "3.40n", "3.40p"]

results = np.empty((len(potentials), len(states), len(energies)))
fit_err = np.empty((len(potentials), len(states), len(energies)))
for dp, potential in enumerate(potentials):
    for ds, state in enumerate(states):
        for de, energy in enumerate(energies):
            theory = np.load(theory_dir / potential / state / f"{energy}.npz")

            if state in skip:
                xs = np.genfromtxt(xs_dir / f"dp_{state[:-1]}_xs_ep.csv", delimiter=',', skip_header=1)
            else:
                xs = np.genfromtxt(xs_dir / f"dp_{state}_xs_ep.csv", delimiter=',', skip_header=1)

            theory_fit = lambda x, A: A * np.interp(x, theory["angle"], theory["cross"])
            params = lmfit.Parameters()
            params.add("A", value=1, min=0)
            lineshape = lmfit.Model(theory_fit)

            fit_weights = np.divide(1, xs[:, 3], out=np.zeros_like(xs[:, 3]), where=xs[:, 3] != 0.0)
            fit = lineshape.fit(params=params, x=xs[:, 0], data=xs[:, 2], weights=fit_weights)

            results[dp, ds, de] = fit.params["A"].value
            fit_err[dp, ds, de] = fit.redchi

In [ ]:
results[1]

In [ ]:
# Choose potential whose reduced chi squared to look at
fit_err[1].round(2)

In [ ]:
# Choose potential to look at
pot_id = 1
for s_fact, err in zip(results[pot_id][:, 1], fit_err[pot_id][:, 1]):
    print(f"{s_fact:.2f}, {err:.2f}")

In [ ]:
# Choose potential to look at
for set in results[0]:
    print(f"{set[1]:.2f}, {(set[0]-set[1]):.2f}, {(set[2]-set[1]):.2f}")

In [ ]:
# Calculate averages
averages = np.average(results, axis=0)
error_prop = np.sqrt(np.sum(fit_err**2, axis=0))

for avg, err in zip(averages[:, 1], error_prop[:, 1]):
    print(f"{avg:.2f}, {err:.2f}")

# for set in averages:
#     print(f"{set[1]:.2f}, {(set[0]-set[1]):.2f}, {(set[2]-set[1]):.2f}")

In [ ]:
# SPE for 0d_3/2
spe_fifth = averages[-1] * 3.40 + (1 - averages[-1]) * 3.89
print(f"{spe_fifth[1]:.2f}, {(spe_fifth[0]-spe_fifth[1]):.2f}, {(spe_fifth[2]-spe_fifth[1]):.2f}")

In [ ]:
# SPE splitting for 0d_5/2
spe_third = averages[2] * 1.78 + (1 - averages[2]) * 3.89
print(f"{spe_third[1]:.2f}, {(spe_third[0]-spe_third[1]):.2f}, {(spe_third[2]-spe_third[1]):.2f}")

In [ ]:
# SO splitting for 0d
so_split = spe_fifth - spe_third
print(f"{so_split[1]:.2f}, {(so_split[0]-so_split[1]):.2f}, {(so_split[2]-so_split[1]):.2f}")